In [2]:
#!pip install requests pandas

In [12]:
import pandas as pd
import talib as ta
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt

In [14]:
df = pd.read_csv('BTCUSDT_3min_2024-05-01_to_2025-05-01.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

In [21]:
# 计算技术指标
window = 7
df['ADX'] = ta.ADX(df['high'], df['low'], df['close'], window)
df['PLUS_DI'] = ta.PLUS_DI(df['high'], df['low'], df['close'], window)
df['MINUS_DI'] = ta.MINUS_DI(df['high'], df['low'], df['close'], window)
df['EMA20'] = ta.EMA(df['close'], 20)
df['SMA50'] = ta.SMA(df['close'], 50)

# 定义策略参数
strategies = {
    'ADX_Strategy': {
        'buy_cond': (df['ADX'] > 25) & (df['PLUS_DI'] > df['MINUS_DI']),
        'sell_cond': (df['ADX'] < 20)
    },
    'DI_Strategy': {
        'buy_cond': (df['PLUS_DI'] > df['MINUS_DI']),
        'sell_cond': (df['PLUS_DI'] < df['MINUS_DI'])
    },
    'EMA_Strategy': {
        'buy_cond': (df['close'] > df['EMA20']),
        'sell_cond': (df['close'] < df['EMA20'])
    },
    'SMA_Strategy': {
        'buy_cond': (df['close'] > df['SMA50']),
        'sell_cond': (df['close'] < df['SMA50'])
    },
    'DI_Crossover_Strategy': {
        'buy_cond': (df['PLUS_DI'] > 25) & (df['MINUS_DI'] < 20),
        'sell_cond': (df['PLUS_DI'] < 20)
    }
}


In [24]:
# 回测引擎函数
def backtest_strategy(df, buy_cond, sell_cond, initial=10000):
    position = 0
    entry_price = 0
    portfolio = [initial]
    trade_returns = []
    peak = initial
    max_drawdown = 0
    
    for i in range(1, len(df)):
        # Buy Signal
        if buy_cond.iloc[i] and position == 0:
            entry_price = df['close'].iloc[i]
            position = 1
        
        # Sell Signal
        elif sell_cond.iloc[i] and position == 1:
            exit_price = df['close'].iloc[i]
            ret = (exit_price - entry_price) / entry_price
            trade_returns.append(ret)
            portfolio_value = portfolio[-1] * (1 + ret)
            portfolio.append(portfolio_value)
            position = 0
        else:
            portfolio.append(portfolio[-1])
        
        # Update max drawdown
        current = portfolio[-1]
        if current > peak:
            peak = current
        drawdown = (peak - current)/peak
        if drawdown > max_drawdown:
            max_drawdown = drawdown
    
    # Calculate performance metrics
    returns = pd.Series(portfolio).pct_change().dropna()
    sharpe = returns.mean() / returns.std() * sqrt(252*24*20)  # 3min data
    
    return {
        'final_value': portfolio[-1],
        'total_return': (portfolio[-1]/initial - 1)*100,
        'sharpe_ratio': sharpe,
        'max_drawdown': max_drawdown*100,
        'num_trades': len(trade_returns),
        'win_rate': len([r for r in trade_returns if r>0])/len(trade_returns)*100 if trade_returns else 0
    }

In [25]:
# 运行所有策略
results = {}
for strategy_name, params in strategies.items():
    results[strategy_name] = backtest_strategy(
        df, 
        params['buy_cond'], 
        params['sell_cond']
    )

# 格式化输出结果
result_df = pd.DataFrame(results).T
result_df.columns = [
    '最终价值', '总收益率 (%)', '夏普比率', 
    '最大回撤 (%)', '交易次数', '胜率 (%)'
]

print("\n策略绩效对比：")
print(result_df.round(2))




策略绩效对比：
                           最终价值  总收益率 (%)  夏普比率  最大回撤 (%)     交易次数  胜率 (%)
ADX_Strategy           13408.64     34.09  0.73     33.39   2849.0   44.54
DI_Strategy             6930.94    -30.69 -0.72     42.55  11363.0   31.91
EMA_Strategy            8345.31    -16.55 -0.27     33.65  12458.0   25.54
SMA_Strategy           11504.51     15.05  0.48     25.19   6950.0   20.32
DI_Crossover_Strategy  10835.06      8.35  0.34     28.13   5781.0   35.01
